## Overview of key results so far

In [ ]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')


# Introduction

In this research project, we are interested in knowing how the future southern ocean carbon sink will be impacted by the interaction of 2 main drivers:

- evolution of atmospheric O$_3$ concentration in the Southern Ocean 
    - 3 different scenarios:
    - no ozone hole ("Fixed Ozone" or "Ozone 1950")
    - ozone hole opens and then recovers ("Hist. Ozone")
    - ozone hole opens around 1990 and then never recovers ("1990 Ozone")
- evolution of atmospheric GHG concentrations
    - 2 scenarios: 
    - SSP1-2.6 (LOW CARBON)
    - SSP 3-7.0 (HIGH CARBON)
    
We force a model (currently PlankTOM12, with a small adaptation to prevent GOC "explosions", see https://nbviewer.org/github/tjarnikova/SOZONE/blob/master/MODPROC_ROBOT/MultiModelMonitor/SANITY_MONITOR.ipynb) with atmospheric forcing from 6 scenarios (combination of the above) of the UKESM1. This forcing has been adapted to work with NEMO (see https://nbviewer.org/github/tjarnikova/SOZONE/tree/master/MO_pipeline/ and https://docs.google.com/document/d/1sK9lCSdDUcnPzPrVHjrkluf2xh09VzTUMCh7yDlOkDk/edit). 

Here I'm keeping track of main results and considerations so far (with pictures!)

# A) Comparison of 3 types of wind forcing

The 3 types of wind forcing we can use to force a NEMO ocean model:

- NCEP- NCEP/NCAR reanalysis (https://gitlab.com/env_greenocean/greenocean_documentation/-/blob/main/HowToCreateNCEPForcingFiles.pdf)
    - /gpfs/data/greenocean/software/products/NCEPForcingData
    - uses CLIO bulk formulation https://www.nemo-ocean.eu/doc/node49.html
    - known wind issues, see docs on gitlab
    - essentially observational
    
- ERA5
    - /gpfs/data/greenocean/software/products/ERA5Forcing
    - https://www.ecmwf.int/en/forecasts/datasets/reanalysis-datasets/era5
    - uses CORE bulk formulation 
    - essentially observational
    - issues around year 1979

- MetOffice
    - processed data is in: /gpfs/data/greenocean/software/resources/MetProcessed/MET_forcing/
    - output of UKESM1 (not observational, has internal weather)
    
The 3 types of forcing are broadly comparable:
https://nbviewer.org/github/tjarnikova/SOZONE/blob/master/prelimEvalForcing/wind_climatology_maps.ipynb

But there are important differences:


In [6]:
# from https://nbviewer.org/github/tjarnikova/SOZONE/blob/master/prelimEvalForcing/wind_climatology_maps.ipynb
pdir = '../uploadPics/windzonal.jpg'
from IPython.display import Image
Image(url= pdir, width=400, height=400)

Basically, in general, NCEP starts out at a lower windspeed than UKESM1/ERA and experiences a stronger increase in winds over 1950-2020 in both summer and winter. Winter winds have increased only slightly in ERA/UKESM1, the increase in NCEp has been stronger. Net magnitudes of wind are also lower.

Importantly/unsettlingly, the 3 different wind forcing products lead to differnt estimations of the total air-sea CO2 flux, both in the total domain and in the southern ocean specifically. A few key points from a summary figure of 4 runs (NCEP, ERA, MET with GOC brakes, MET without GOC brakes):

- The NCEP run (GAO1, blue, using CLIO) has somewhat lower uptake than the others, both globally and in SO specifically. 
    - this lower uptake is despite lower winds overall: high winds in SO should lead to lower carbon sink. 
    - on the other hand, the NCEP wind product is the only one that sees an increase in SO winds in winter, which could be the cause of a slowdown
    - or it could have something to do with using CLIO instead of CORE.
- The other 3 runs, using CORE and comparable though independent wind products, tend to have similar estimates of CO2 flux, both globally and in the SO. 
    - the "weirdness" in the first 30 years or so of the air-sea co2 flux in the yellow met office run is partly because this run is not spun up - will be fixe

In [5]:
pdir = '../uploadPics/co2flux_era_met_ncep.jpg'
from IPython.display import Image
Image(url= pdir, width=700, height=400)


In [5]:
pdir = '../uploadPics/wspd_10yrmean.jpg'
from IPython.display import Image
Image(url= pdir, width=700, height=400)